In [1]:
from typing import Tuple, List
from matrix import Matrix

In [2]:
m1 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
m1_t = m1.transpose()
aat = m1.multiply(m1_t) # a x at
ata = m1_t.multiply(m1) # at x a

In [158]:
def power_iteration(matrix: 'Matrix', max_iter=1000, tol=1e-10) -> Tuple[float, List[float]]:
    n = matrix.rows
    b_k = [1] * n  # Initial guess for the eigenvector

    for _ in range(max_iter):
        # Multiply matrix by the vector
        b_k1 = [sum(matrix.get(i, j) * b_k[j] for j in range(n)) for i in range(n)]
        
        # Normalize the resulting vector
        norm = sum(b_k1[i] ** 2 for i in range(n)) ** 0.5
        b_k1 = [x / norm for x in b_k1]

        # Check for convergence
        if sum((b_k1[i] - b_k[i]) ** 2 for i in range(n)) < tol:
            break

        b_k = b_k1

    # Rayleigh quotient gives an approximation of the eigenvalue
    eigenvalue = sum(b_k[i] * sum(matrix.get(i, j) * b_k[j] for j in range(n)) for i in range(n))
    return eigenvalue, b_k


In [159]:
def deflate(matrix: 'Matrix', eigenvalue: float, eigenvector: List[float]) -> 'Matrix':
    n = matrix.rows
    outer_product = [
        [eigenvector[i] * eigenvector[j] for j in range(n)]
        for i in range(n)
    ]
    return matrix.add(Matrix([[-eigenvalue * outer_product[i][j] for j in range(n)] for i in range(n)]))


In [160]:
def eigen_decomposition(matrix: 'Matrix', num_eigenvalues: int) -> Tuple[List[float], List[List[float]]]:
    eigenvalues = []
    eigenvectors = []
    current_matrix = matrix

    for _ in range(num_eigenvalues):
        eigenvalue, eigenvector = power_iteration(current_matrix)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)
        current_matrix = deflate(current_matrix, eigenvalue, eigenvector)

    print(eigenvalues, eigenvectors)
    
    return eigenvalues, eigenvectors


In [161]:
def compute_singular_values(ata_matrix: 'Matrix') -> List[float]:
    eigenvalues, _ = eigen_decomposition(ata_matrix, ata_matrix.rows)
    return [eigenvalue ** 0.5 if eigenvalue >= 0 else 0 for eigenvalue in eigenvalues]


In [162]:
def normalize_vector(vector: List[float]) -> List[float]:
    # Calculate the magnitude (Euclidean norm) of the vector
    magnitude = sum(x ** 2 for x in vector) ** 0.5
    
    # Avoid division by zero if the magnitude is zero
    if magnitude == 0:
        raise ValueError("Cannot normalize a zero vector.")
    
    # Divide each component of the vector by the magnitude
    return [x / magnitude for x in vector]

In [163]:
from typing import List

def dot_product(v1: List[float], v2: List[float]) -> float:
    """Helper function to compute the dot product of two vectors."""
    return sum(x * y for x, y in zip(v1, v2))

def scalar_multiply(scalar: float, vector: List[float]) -> List[float]:
    """Helper function to multiply a vector by a scalar."""
    return [scalar * x for x in vector]

def vector_subtract(v1: List[float], v2: List[float]) -> List[float]:
    """Helper function to subtract two vectors."""
    return [x - y for x, y in zip(v1, v2)]

def normalize_vector(vector: List[float]) -> List[float]:
    """Normalize a vector to have unit length."""
    magnitude = (sum(x ** 2 for x in vector)) ** 0.5
    if magnitude == 0:
        raise ValueError("Cannot normalize a zero vector.")
    return [x / magnitude for x in vector]

def gram_schmidt(vectors: List[List[float]]) -> List[List[float]]:
    """Apply Gram-Schmidt process to make a set of vectors orthonormal."""
    orthonormal_vectors = []
    
    for v in vectors:
        # Start with the current vector
        u = v
        
        # Subtract the projection of v onto each previously computed u_j
        for u_prev in orthonormal_vectors:
            projection = scalar_multiply(dot_product(v, u_prev) / dot_product(u_prev, u_prev), u_prev)
            u = vector_subtract(u, projection)
        
        # Normalize the resulting vector
        u = normalize_vector(u)
        
        # Add the orthonormalized vector to the list
        orthonormal_vectors.append(u)
    
    return orthonormal_vectors


In [164]:
def svd(a: 'Matrix') -> Tuple['Matrix', 'Matrix', 'Matrix']:
    a_t = a.transpose()
    aat = a.multiply(a_t) # a x at
    ata = a_t.multiply(a)

    # Step 2: Compute singular values
    singular_values = compute_singular_values(aat) 

    # Step 3: Eigenvector decomposition to get U and V
    _, U_vectors = eigen_decomposition(aat, aat.rows)
    _, V_vectors = eigen_decomposition(ata, ata.cols)

    U_vectors = gram_schmidt(U_vectors)
    
    # Step 4: Construct U, Sigma, V^T
    U = Matrix(U_vectors)
    Sigma = Matrix([[singular_values[i] if i == j else 0 for j in range(a.cols)] for i in range(a.rows)])
    V = Matrix(V_vectors)
    
    return U, Sigma, V.transpose()


In [165]:
a = Matrix([[1,0,0], [0,3,-1], [0,-1,3]])
# u, s, v_t = svd(m1)
u, s, v_t = svd(a)

[3.9999999996507536, 1.0000000013969839, -2.0956045374773395e-09] [[1.0789593218160839e-05, 0.7071067811453885, 0.7071067811453885], [0.9999999990686135, -3.0518625906418764e-05, -3.0518625906418764e-05], [-0.5773266233965968, 0.5773620917229864, 0.5773620917229864]]
[3.9999999996507536, 1.0000000013969839, -2.0956045374773395e-09] [[1.0789593218160839e-05, 0.7071067811453885, 0.7071067811453885], [0.9999999990686135, -3.0518625906418764e-05, -3.0518625906418764e-05], [-0.5773266233965968, 0.5773620917229864, 0.5773620917229864]]
[3.9999999996507536, 1.0000000013969839, -2.0956045374773395e-09] [[1.0789593218160839e-05, 0.7071067811453885, 0.7071067811453885], [0.9999999990686135, -3.0518625906418764e-05, -3.0518625906418764e-05], [-0.5773266233965968, 0.5773620917229864, 0.5773620917229864]]


In [166]:
print(u), print(s), print(v_t)

1.0789593218160839e-05	0.7071067811453885	0.7071067811453885
0.9999999999417923	-7.6293945308059176e-06	-7.6293945308059176e-06
0.0	0.7071067811865475	0.7071067811865475
1.9999999999126883	0	0
0	1.000000000698492	0
0	0	0
1.0789593218160839e-05	0.9999999990686135	-0.5773266233965968
0.7071067811453885	-3.0518625906418764e-05	0.5773620917229864
0.7071067811453885	-3.0518625906418764e-05	0.5773620917229864


(None, None, None)

In [167]:
print(u.multiply(s).multiply(v_t))

0.500000000523869	-7.4092946038398e-10	0.408244192279933
1.618438982158892e-05	1.9999999981663386	-1.1546576515987645
0.500000000320142	-2.1579927345997557e-05	0.408256650542537


In [154]:
A

[[14, 32, 50], [32, 77, 122], [50, 122, 194]]